In [1]:
import preprocessing as pp
import pandas as pd
import numpy as np

In [2]:
customer_df, candidate_df, supplier_df, vehicle_df, distance_w_to_s_df, distance_w_to_c_df, demand_cus_period_df, demand_cus_period_scene_df = pp.read_and_prep_data()

# 2 methods for clustering

customer_df, cluster_center_df = pp.ori_cluster_by_cus_loc(customer_df, n_clusters=100, random_state=42, n_init=10)
# customer_df, cluster_center_df = pp.const_cluster_by_cus_loc(customer_df, n_clusters=100, size_min=4, size_max=400, random_state=42)

agg_dem_cus_period_df = pp.agg_dem_cus_period(demand_cus_period_df,customer_df)

In [ ]:
# def create_dis_mat_df(depart_df:pd.DataFrame, arrive_df:pd.DataFrame, method: 'cityblock'):
#     # Assuming warehouse_df and customer_df have columns ['x', 'y']
#     depart_coords = depart_df[['CandidateEasting', 'CandidateNorthing']].values
#     arrive_coords = arrive_df[['CustomerEasting', 'CustomerNorthing']].values

#     # Compute Euclidean distance matrix
#     distance_matrix = cdist(depart_coords, arrive_coords, metric=method)/1000
#     # display(distance_matrix)
#     # Convert to a Pandas DataFrame for better readability
#     distance_df = pd.DataFrame(distance_matrix, 
#                             index=depart_df.index, 
#                             columns=arrive_df.index)

#     return distance_df

# def calculate_cost_from_w_to_cluster(distance_df:pd.DataFrame, vehicle_df:pd.DataFrame):
#     cost_w_to_cluster = distance_df.copy()
#     for w in range(len(distance_df.index)):
#         for c in range(len(distance_df.columns)):
#             cost_w_to_cluster.at[w,c] = 2 * distance_df.at[w,c] * vehicle_df.at[2,'VehicleCostPerMileAndTonneOverall'] / 1000   
#     return cost_w_to_cluster

# def calculate_cost_from_w_to_s(distance_df:pd.DataFrame, vehicle_df:pd.DataFrame, supplier_df:pd.DataFrame):
#     cost_w_to_s = distance_df.copy()
#     for w in range(len(distance_df.index)):
#         for s in range(len(distance_df.columns)):
#             if supplier_df.at[s,'SupplierVehicleType'] == 0:
#                 cost_w_to_s.at[w,s] = 2 * distance_df.at[w,s] * vehicle_df.at[0,'VehicleCostPerMileAndTonneOverall'] / 1000   
#             elif supplier_df.at[s,'SupplierVehicleType'] == 1:
#                 cost_w_to_s.at[w,s] = 2 * distance_df.at[w,s] * vehicle_df.at[1,'VehicleCostPerMileAndTonneOverall'] / 1000   
#             else:
#                 pass
#     return cost_w_to_s

In [ ]:
distance_w_to_cluster_df = pp.create_dis_mat_df(candidate_df, cluster_center_df,'cityblock')

# Create Cost
cost_w_to_cluster = pp.calculate_cost_from_w_to_cluster(distance_w_to_cluster_df, vehicle_df)
cost_w_to_s = pp.calculate_cost_from_w_to_s(distance_w_to_s_df, vehicle_df, supplier_df)

In [ ]:
asdfasdf